In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os
import glob
import numpy as np
import pandas as pd

## MLP 128 x 128 (no resize)

### 1. Split Dataset

In [4]:
current = os.getcwd()
data_folder = os.path.join(current, "normalized_images_no_resize_128")
chess_types_folders = glob.glob(os.path.join(data_folder, "*"))
pieces_info = []
labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}
for chess_types in chess_types_folders:
    pieces = glob.glob(f'{chess_types}/*')
    # print(pieces)
    type = chess_types.split("/")[-1]
    for piece in pieces:
        p = {"normalized_img": np.load(piece).reshape(-1), "label": labels[type]}
        pieces_info.append(p)
chess_df = pd.DataFrame(pieces_info)

In [9]:
chess_1 = chess_df[chess_df['label'] == 1]
chess_2 = chess_df[chess_df['label'] == 2]
chess_3 = chess_df[chess_df['label'] == 3]
chess_4 = chess_df[chess_df['label'] == 4]
chess_5 = chess_df[chess_df['label'] == 5]
chess_6 = chess_df[chess_df['label'] == 6]

train_df_1, test_df_1 = train_test_split(chess_1, test_size=0.3, random_state=1)
train_df_2, test_df_2 = train_test_split(chess_2, test_size=0.3, random_state=1)
train_df_3, test_df_3 = train_test_split(chess_3, test_size=0.3, random_state=1)
train_df_4, test_df_4 = train_test_split(chess_4, test_size=0.3, random_state=1)
train_df_5, test_df_5 = train_test_split(chess_5, test_size=0.3, random_state=1)
train_df_6, test_df_6 = train_test_split(chess_6, test_size=0.3, random_state=1)

train_dfs_list = [train_df_1, train_df_2, train_df_3, train_df_4, train_df_5, train_df_6]
test_dfs_list = [test_df_1, test_df_2, test_df_3, test_df_4, test_df_5, test_df_6]

train_df = pd.concat(train_dfs_list, ignore_index=True)
test_df = pd.concat(test_dfs_list, ignore_index=True)

# Load dataset
X_train, y_train = np.array(train_df['normalized_img']), np.array(train_df['label'])
X_test, y_test = np.array(test_df['normalized_img']), np.array(test_df['label'])

X_train = np.array([x for x in X_train])
X_test = np.array([x for x in X_test])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test.shape

(170, 50176)

### 2. MLP

In [13]:
mlp = MLPClassifier(hidden_layer_sizes=(1024, 512, 128), activation='relu', solver='adam', max_iter=500, random_state=1)
mlp.fit(X_train, y_train)

# Predict on test set
y_test_pred = mlp.predict(X_test)

final_test_error = 1 - accuracy_score(y_test, y_test_pred)

accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')  # Use 'macro' or 'weighted' for multi-class
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Test Error: {final_test_error:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))
# labels = {"King": 1, "Knight":2, "Bishop":3, "Rook":4, "Pawn":5, "Queen":6}


Test Error: 0.6412
Accuracy: 0.3588
Precision: 0.3526
Recall: 0.3588
F1 Score: 0.3525

Confusion Matrix:
[[ 8  1  6  2  1  5]
 [ 3 18  2  7  2  0]
 [ 7  7  7  1  3  2]
 [ 3  4  2 13  7  2]
 [ 3  3  6  7 11  3]
 [ 5  6  3  2  4  4]]

Classification Report:
              precision    recall  f1-score   support

           1       0.28      0.35      0.31        23
           2       0.46      0.56      0.51        32
           3       0.27      0.26      0.26        27
           4       0.41      0.42      0.41        31
           5       0.39      0.33      0.36        33
           6       0.25      0.17      0.20        24

    accuracy                           0.36       170
   macro avg       0.34      0.35      0.34       170
weighted avg       0.35      0.36      0.35       170

